In [2]:
spark

res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7cabb261


In [22]:
spark.version

res11: String = 2.3.1


In [23]:
val input = sc.textFile("Scorecard.csv").map(line =>line.split(","))

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[15] at map at <console>:25


In [24]:
val names = input.map( x => (x(4)))

names: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[16] at map at <console>:27


In [25]:
names.take(5)

res12: Array[String] = Array(INSTNM, COMMUNITY COLLEGE OF THE AIR FORCE, ALABAMA A & M UNIVERSITY, UNIVERSITY OF ALABAMA AT BIRMINGHAM, ALABAMA AVIATION AND TECHNICAL COLLEGE)
